In [11]:
#importing stuff
import numpy as np
import pandas as pd
from numpy import array
from numpy import argmax
from collections import Counter

In [12]:
#dictionary of state codes used for cleaning
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'BC': 'British Columbia',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming',
        'MH': 'Marshal Islands'
}

In [13]:
#numeric encoder
def nothotter(x):
    d = {}
    n = len(x)
    for a in range(0, n):
        d[x[a]] = a
    return d

In [14]:
#one hot encoding
def onehotter(df,pre):
    df= pd.get_dummies(df,prefix=pre)
    return d

In [15]:
import operator
def topper(data, col):
    if(col == 'country_of_citizenship'):
        u = 13
    elif(col == 'employer_decl_info_title'):
        u = 23
    x = data.loc[data['case_status'] == 1]
    d = Counter(x[col])
    sorted_x = sorted(d.items(), key=operator.itemgetter(1))
    sorted_x.reverse()
    top = sorted_x[:u]
    l = []
    for x in top:
        l.append(x[0])
    data.loc[~data[col].isin(l),col] = 'Others'
    return data

In [16]:
def encoder(df):
    oh = ['country_of_citizenship', 'employer_decl_info_title', 'employer_state', 'foreign_worker_info_education']
    nh = ['employer_city', 'foreign_worker_info_inst', 'foreign_worker_info_major', 'pw_soc_title']
    no = [
 'agent_firm_name',
 'case_status',
 'class_of_admission',
 'employer_name',
 'employer_num_employees',
 'pw_level_9089',
 'wage_offer_from_9089']
    
    #certified/denied
    df.loc[df['case_status'] == 'Certified', 'case_status'] = 1
    df.loc[df['case_status'] == 'Denied' ,'case_status'] = 0
    
    #take top 10 countries and decl title and make everything others
    df = topper(df, 'country_of_citizenship')
    df = topper(df, 'employer_decl_info_title')
    
    #taking subset for hotting
    df_hot=pd.DataFrame()
    for x in oh:
        df_hot[x]=df[x]
        
    #hotting
    df_hot = pd.get_dummies(df_hot,prefix=oh)
    
    #taking subset for not hotting
    df_not=pd.DataFrame()
    for x in nh:
        df_not[x]=df[x]
    
    #notting
    for x in nh:
        d = nothotter(df_not[x].unique())
        df_not[x] = df_not[x].apply(lambda x: d[x])
    
    
    #Unioning:
    dx = pd.DataFrame()
    for x in df_hot.columns:
        dx[x] = df_hot[x]
    for x in nh:
        dx[x] = df_not[x]
    for x in no:
        dx[x] = df[x]
    return dx

In [17]:
data = pd.read_csv('../Dataset/dweepafinal.csv')
def statemaker(x):
    try:
        y = len(x)
    except:
        y = -1
    if(y<3 and y!=-1):
        return states[x].lower()
    elif y>2:
        return x.lower()
    return x
data['employer_state'] = data['employer_state'].apply(statemaker)
data.drop(['foreign_worker_info_state'],axis=1,inplace=True)
data.drop(['Unnamed: 0'], axis = 1, inplace = True)
new_encoder = encoder(data)

In [18]:
agents  = new_encoder['employer_name'].unique()
agents_success = {}

for a in agents:
    try:
        x = Counter(new_encoder['employer_name']==a)
        y = Counter(new_encoder[new_encoder['employer_name']==a]['case_status']==1)
        try:
            c = y[True]
        except:
            c = 0
        agents_success[a] = c/x[True]
        if x[True]<10:
            agents_success[a] = 0.5
    except:
        continue

def get_success(x):
    try:
        y = agents_success[x]
    except:
        y = 0
    return y

new_encoder['employer_success'] = new_encoder['employer_name'].apply(get_success)


agents  = new_encoder['agent_firm_name'].unique()
agents_success = {}

for a in agents:
    try:
        x = Counter(new_encoder['agent_firm_name']==a)
        y = Counter(new_encoder[new_encoder['agent_firm_name']==a]['case_status']==1)
        try:
            c = y[True]
        except:
            c = 0
        agents_success[a] = c/x[True]
        if x[True]<10:
            agents_success[a] = 0.5
    except:
        continue
        
new_encoder['agent_success'] = new_encoder['agent_firm_name'].apply(get_success)


new_encoder.drop(['employer_name', 'agent_firm_name'], axis = 1, inplace = True)

In [19]:
new_encoder.to_csv("../Dataset/encoded.csv")

AttributeError: 'DataFrame' object has no attribute 'to_cv'